In [17]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table as dt
from dash.dependencies import Input, Output, State
import requests
import textblob
import plotly.express as px

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


def get_pushshift_data(data_type, **kwargs):
    print("I am in get_pushshift_data")
    base_url = f"https://api.pushshift.io/reddit/search/{data_type}/"
    print(base_url)
    payload = kwargs
    print(payload)
    request = requests.get(base_url, params=payload)
    return request.json()

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
application = app.server


app.layout = html.Div([
    html.Div([
    html.H2("RegAg Analysis Portal"),
    dcc.Input(id='input_1_state', value='regenerative agriculture', type='text'),
    #dcc.Dropdown(id='dropdown_1', options=[{'label': i, 'value': i} for i in ['regenerative agriculture', 'organic']], 
      #           value = 'regenerative agriculture'),
             
    html.H5("Choose number of days for which the data will be collected for?"),
    dcc.Dropdown(id='dropdown_2', options=[{'label': 'A Week Ago', 'value': '7d'}, {'label': ' 2 Weeks Ago', 'value': '14d'},{'label':' A Year Ago','value':'1y'}], 
                 value = '7d'),   
    html.Button(id='submit-button', children='Submit'),
        
    ],style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw', 'width':'5'}),
    
    html.Div([
    html.Div(id="table1"),
    dcc.Graph(id='graph1')
], style={'display': 'inline-block', 'vertical-align': 'top','margin-left': '3vw', 'margin-top': '3vw', 'width':'5'}) 
])

#], style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw', 'margin-bottom':'3vw', 'margin-right':'3vw'} ) 



@app.callback(Output('table1','children'),
              Output('graph1', 'figure'),
              
              [Input('input_1_state', 'value')],
              [Input('dropdown_2', 'value')],
              [State('submit-button','n_clicks')]
              )

def get_new_data(input_1_state_value,dropdown_2_value,n_clicks):
    print("I am in get_new_data")
    print(input_1_state_value)
    print(dropdown_2_value)
    json_data=get_pushshift_data(data_type="comment", q=input_1_state_value, after=dropdown_2_value, aggs="subreddit", size=1000)
    print("I am in json_data")
    data=json_data.get("data")
    df = pd.DataFrame(data)
    df['count'] = 1
    min_df = df[['subreddit', 'count']]
    grouped_df = min_df.groupby(['subreddit']).sum()
    grouped_df = grouped_df.sort_values(by=['count'], ascending=False)[0:10]
    grouped_df = grouped_df.reset_index()
    grouped_df.head() #top 5 subreddit where the phrase appears most frequently
    columns =  [{"name": i, "id": i,} for i in (grouped_df.columns)]
    
    #################
   

    fig= px.bar(grouped_df,              # our dataframe
       x="subreddit",         # x will be the 'key' column of the dataframe
       y="count",   # y will be the 'doc_count' column of the dataframe
       title=f"Subreddits with " + input_1_state_value + " activity",
       labels={"count": "# comments","subreddit": "Subreddits"}, # the axis names
       color_discrete_sequence=["blueviolet"], # the colors used
       height=500,
       width=800)
    #################
    return dt.DataTable(data=grouped_df.to_dict('rows'), columns=columns), fig
            
        
if __name__ == '__main__':
    application.run(debug=False, port=8080)        

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Feb/2021 21:58:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2021 21:58:56] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/Feb/2021 21:58:56] "GET /_dash-dependencies HTTP/1.1" 200 -


I am in get_new_data
regenerative agriculture
7d
I am in get_pushshift_data
https://api.pushshift.io/reddit/search/comment/
{'q': 'regenerative agriculture', 'after': '7d', 'aggs': 'subreddit', 'size': 1000}


127.0.0.1 - - [03/Feb/2021 21:58:58] "POST /_dash-update-component HTTP/1.1" 200 -


I am in json_data
